# On trigger signal acquisition

## Description

This example shows how to acquire 16k samples of signal on fast analog inputs.
Signal will be acquired when the internal trigger condition is meet.
Time length of the acquired signal depends on the time scale of a buffer
that can be set with a decimation factor.

TODO: describe some calculations

## Required hardware

- Red Pitaya
- Signal (function) generator

![wiring diagram](img/on_given_trigger_acquire_signal_on_fast_analog_input.png)

The `rp` object is an instance of the `redpitaya` class.
When the object is initialized, the FPGA bitstream is loaded and
memory mapped FPGA registers are mapped into software.
Repeating FPGA bitstream loading will cause all registers to reset,
while mapping the memory space multiple times will cause segmentation faults.
So untill this issues are handled by the driver
a `redpitaya` instance should be created only once.

In [1]:
import time

# this will load the FPGA
from mercury import overlay
o = overlay("mercury")
# wait a bit for the overlay to be properly applied
# TODO it should be automated in the library
time.sleep(0.5)

ResourceWarning: Requested overlay is already loaded.

In [2]:
# oscilloscope
from mercury import osc
# instantiate both oscilloscopes
channels = (0, 1)
oscs = [osc(ch, 1.0) for ch in channels]

In [3]:
# acquisition configuration
size = 1024

for ch in channels:
    # TODO: for now bypass input filter
    oscs[ch].filter_bypass = True
    
    # decimation rate
    oscs[ch].decimation = 1

    # trigger timing
    oscs[ch].trigger_pre_delay  = size//4*1
    oscs[ch].trigger_post_delay = size//4*3

    # trigger level [V], edge and holdoff [periods]
    oscs[ch].level = [-0.1, +0.1]
    oscs[ch].edg = 'pos'
    oscs[ch].holdoff = 20

    # trigger source mask
    mask = [0x1<<10, 0x2<<10, 0x4<<10]
    mask[2] |= 0x8<<10
    oscs[ch].mask = mask

    oscs[ch].reset()

oscs[0].start()
#oscs[0].trigger()
while oscs[0].status_run(): pass
print('triggered')

# read data from FPGA FIFO into memory and display it
buff = [oscs[ch].data(size) for ch in channels];

triggered


In [4]:
import time
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool, Range1d
from bokeh.plotting import figure
from bokeh.layouts import widgetbox
from bokeh.resources import INLINE 
#output_notebook(resources=INLINE)
output_notebook()

Loading BokehJS ...

In [5]:
x = (np.arange(size) - oscs[ch].trigger_pre_delay) / oscs[0].FS

colors = ('red', 'blue')
tools = "pan,wheel_zoom,box_zoom,reset,crosshair"
p = figure(plot_height=500, plot_width=900, title="oscilloscope", toolbar_location="above", tools=(tools))
p.xaxis.axis_label='time [s]'
p.yaxis.axis_label='voltage [V]'
p.y_range=Range1d(-1.2, +1.2)
r = [p.line(x, buff[i], line_width=1, line_alpha=0.7, color=colors[i]) for i in channels]

# trigger time/amplitude
ch = 0
if oscs[ch].edg is 'pos': level = oscs[ch].level[1]
else                    : level = oscs[ch].level[0]
h_trigger_t = [p.line ([0,0], [-1.2, +1.2], color="black", line_width=1, line_alpha=0.75),
               p.quad(left=[0], right=[oscs[ch].holdoff*oscs[ch].sample_period],
                      bottom=[-1.2], top=[+1.2], color="grey", alpha=0.25)]
h_trigger_a = [p.line ([x[0], x[-1]], [level]*2, color="black", line_width=1, line_alpha=0.75),
               p.quad(bottom=[oscs[ch].level[0]], top=[oscs[ch].level[1]],
                      left=[x[0]], right=[x[-1]], color="grey", alpha=0.25)]

# configure hover tool
hover = HoverTool(mode = 'vline', tooltips=[("T", "@x"), ("V", "@y")], renderers=r)
p.add_tools(hover)

# get an explicit handle to update the next show cell with
target = show(p, notebook_handle=True)

In [ ]:
while True:
    oscs[0].reset()
    oscs[0].start()
    while oscs[0].status_run(): pass
    buff = [oscs[ch].data(size) for ch in channels]
#    buff = np.absolute(np.fft.fft(buff))
    for ch in channels:
        r[ch].data_source.data['y'] = buff[ch]
    # push updates to the plot continuously using the handle (intererrupt the notebook kernel to stop)
    push_notebook(handle=target)
#    time.sleep(0.05)

In [ ]:
oscs[0].show_regset()

In [ ]:
del(oscs)
del(gen0)
del(o)


In [13]:
from IPython.display import display
from ipywidgets import *

slider = FloatRangeSlider(value=[-0.1,+0.1], min=-1.2, max=+1.2, step=0.02, description='trigger level')
def on_value_change(change):
    h_trigger_a[0].data_source.data['y']      = [change['new'][1]]*2
    h_trigger_a[1].data_source.data['bottom'] = [change['new'][0]]
    h_trigger_a[1].data_source.data['top']    = [change['new'][1]]
    push_notebook(handle=target)

slider.observe(on_value_change, names='value')
display (slider)

In [ ]:
class oscilloscope (object):
    def __init__ (self, ):